# Table 1 Replication - Leverage Analysis (1965-2003)

This notebook replicates **Table 1** from the paper, showing descriptive statistics for **All Firms** and **Survivors** using Compustat data.

---


In [2]:
# STEP 0 — Environment Setup
# Install required packages: wrds, pandas, numpy, scipy, jupyter, ipykernel
%pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libraries
import wrds
import pandas as pd
import numpy as np
from scipy import stats
import os

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
pd.set_option('display.width', 120)


## STEP 1 — Connect to WRDS

Establish connection to WRDS database. You'll be prompted for username and password.


In [ ]:
# Connect to WRDS (enter credentials when prompted)
db = wrds.Connection()


In [ ]:
# Sanity check: verify 'comp' library is available
libraries = db.list_libraries()
print(f"Available libraries: {len(libraries)}")
print(f"'comp' available: {'comp' in libraries}")


## STEP 2 — Pull Compustat Data

Pull annual Compustat data matching Section I of the paper:
- **Nonfinancial firms** (SIC codes outside 6000-6999)
- 1965–2003
- Consolidated, domestic, INDL format


In [ ]:
# STEP 2 — Pull Compustat Data
# Pull annual Compustat data matching Section I: nonfinancial firms, 1965-2003, consolidated domestic
# Note: Excludes financial firms (SIC 6000-6999) directly in SQL

# Option: Load existing data to skip download
if os.path.exists('data/01_raw_data.csv'):
    print("Loading existing raw data from data/01_raw_data.csv")
    df = pd.read_csv('data/01_raw_data.csv', parse_dates=['datadate'])
    print(f"Loaded {len(df):,} observations")
else:
    print("No existing data found, downloading from WRDS...")

    sql = """
    SELECT
        gvkey,
        datadate,
        fyear,
        sich,      -- SIC code (historical SIC code column in funda table)
        at,        -- total assets
        dlc,       -- short-term debt
        dltt,      -- long-term debt
        sale,
        oibdp,
        ppent,
        prcc_f,
        csho,
        pstkl,
        txditc,
        intan,
        dvc
    FROM comp.funda
    WHERE indfmt = 'INDL'
      AND datafmt = 'STD'
      AND popsrc = 'D'
      AND consol = 'C'
      AND fyear BETWEEN 1965 AND 2003
      AND sich IS NOT NULL
      AND (sich < 6000 OR sich > 6999)
    """

    print("Downloading data from WRDS... (this may take a few minutes)")
    df = db.raw_sql(sql, date_cols=['datadate'])
    print(f"Downloaded {len(df):,} observations")
    print(f"Unique firms (gvkeys): {df['gvkey'].nunique():,}")


In [ ]:
# Additional Python-side filter for financials (ensures exclusion even when loading from CSV)
df = df[df['sich'].notna()]
df = df[(df['sich'] < 6000) | (df['sich'] > 6999)]
print(f"After excluding financials (SIC 6000-6999): {len(df):,} observations")
print(f"Unique firms: {df['gvkey'].nunique():,}")


In [ ]:
# Save raw downloaded data
os.makedirs('data', exist_ok=True)
df.to_csv('data/01_raw_data.csv', index=False)
print(f"✓ Saved raw data: {len(df):,} observations to data/01_raw_data.csv")


In [ ]:
# Preview the raw data
df.head()


## STEP 3 — Basic Cleaning

Apply paper's cleaning rules:
1. Require non-missing, positive assets
2. Fill missing debt components with zero
3. Calculate total debt


In [ ]:
print(f"Before cleaning: {len(df):,} observations")

# Require non-missing, positive assets
df = df[df['at'].notna() & (df['at'] > 0)]
print(f"After asset filter: {len(df):,} observations")

# Replace missing debt components with 0
df['dlc'] = df['dlc'].fillna(0)
df['dltt'] = df['dltt'].fillna(0)

# Total debt
df['debt'] = df['dlc'] + df['dltt']

print(f"Debt computed for all {len(df):,} observations")


In [ ]:
# Save cleaned data
df.to_csv('data/02_cleaned_data.csv', index=False)
print(f"✓ Saved cleaned data: {len(df):,} observations to data/02_cleaned_data.csv")


## STEP 4 — Construct Leverage Measures

Calculate book and market leverage as defined in the Appendix:
- **Book leverage** = Total Debt / Total Assets
- **Market leverage** = Total Debt / (Total Debt + Market Equity)


In [ ]:
# Book leverage
df['book_lev'] = df['debt'] / df['at']

# Market equity
df['me'] = df['prcc_f'] * df['csho']

# Market leverage
df['market_lev'] = df['debt'] / (df['debt'] + df['me'])

print(f"Before leverage filter: {len(df):,} observations")

# Keep leverage in [0,1]
df = df[
    (df['book_lev'].between(0, 1)) &
    (df['market_lev'].between(0, 1))
]

print(f"After leverage filter [0,1]: {len(df):,} observations")


In [ ]:
# Save leverage data
df.to_csv('data/03_leverage_data.csv', index=False)
print(f"✓ Saved leverage data: {len(df):,} observations to data/03_leverage_data.csv")


## STEP 5 — Construct Table 1 Variables

Create all variables that appear in Table 1:
- Log sales (firm size) — set to NaN for non-positive sales
- Market-to-book ratio
- Profitability
- Tangibility
- Intangibles
- Dividend payer dummy


In [ ]:
# Log sales (proxy for firm size)
# Set to NaN for non-positive sales (don't clip to 1, that creates fake values)
df['log_sales'] = np.where(df['sale'] > 0, np.log(df['sale']), np.nan)

# Market-to-book ratio
df['mtb'] = (
    df['me']
    + df['debt']
    + df['pstkl'].fillna(0)
    - df['txditc'].fillna(0)
) / df['at']

# Profitability (EBITDA / Assets)
df['profitability'] = df['oibdp'] / df['at']

# Tangibility (PPE / Assets)
df['tangibility'] = df['ppent'] / df['at']

# Intangibles (Intangible Assets / Assets)
df['intangibles'] = df['intan'] / df['at']

# Dividend payer (binary: 1 if pays dividend, 0 otherwise)
df['div_payer'] = (df['dvc'].fillna(0) > 0).astype(int)

print("All Table 1 variables constructed")
print(f"log_sales non-missing: {df['log_sales'].notna().sum():,} ({100*df['log_sales'].notna().mean():.1f}%)")


In [ ]:
# Save variables data
df.to_csv('data/04_variables_data.csv', index=False)
print(f"✓ Saved variables data: {len(df):,} observations to data/04_variables_data.csv")


## STEP 6 — Cash-Flow Volatility

Calculate rolling 3-year standard deviation of **scaled** operating income (oibdp/at).

Using raw oibdp leads to scale issues (large firms have large volatility in dollar terms). Scaling by assets makes this a profitability-type measure.


In [ ]:
# Sort by firm and year
df = df.sort_values(['gvkey', 'fyear'])

# Scaled cash flow (oibdp / assets)
df['cf_base'] = df['oibdp'] / df['at']

# Rolling 3-year standard deviation of SCALED operating income
df['cf_vol'] = (
    df.groupby('gvkey')['cf_base']
      .rolling(window=3, min_periods=3)
      .std()
      .reset_index(level=0, drop=True)
)

print(f"Cash-flow volatility computed for {df['cf_vol'].notna().sum():,} observations")
print(f"cf_vol summary: mean={df['cf_vol'].mean():.4f}, median={df['cf_vol'].median():.4f}, std={df['cf_vol'].std():.4f}")


## STEP 7 — Industry Median Book Leverage

Calculate industry median leverage using 2-digit SIC codes.

**Note:** This uses SIC-2 classification. The paper may use Fama-French 38 industry classification, which could lead to minor differences in `ind_med_lev`.


In [ ]:
# Create 2-digit SIC code (using sich column from Compustat)
df['sic2'] = df['sich'] // 100

# Industry median leverage (by SIC-2 and year)
df['ind_med_lev'] = (
    df.groupby(['sic2', 'fyear'])['book_lev']
      .transform('median')
)

print(f"Industry median leverage computed")
print(f"Number of unique industries (SIC-2): {df['sic2'].nunique()}")


In [ ]:
# Save final data (no trimming applied to rows - trimming done in summary stats)
df.to_csv('data/05_final_data.csv', index=False)
print(f"✓ Saved final data: {len(df):,} observations to data/05_final_data.csv")


## STEP 8 — Define Survivors

**Survivors** are defined as firms with ≥20 years of book leverage data in the sample.


In [ ]:
# Count years of book leverage data per firm
lev_counts = df.groupby('gvkey')['book_lev'].count()

# Survivors: firms with ≥20 years
survivors = lev_counts[lev_counts >= 20].index

total_firms = df['gvkey'].nunique()
print(f"Total unique firms: {total_firms:,}")
print(f"Survivors (≥20 years): {len(survivors):,}")
if total_firms > 0:
    print(f"Survivor rate: {100 * len(survivors) / total_firms:.1f}%")
else:
    print("Survivor rate: N/A (no firms in dataset)")

# Create two datasets
df_all = df.copy()
df_surv = df[df['gvkey'].isin(survivors)].copy()

print(f"\nAll Firms dataset: {len(df_all):,} observations")
print(f"Survivors dataset: {len(df_surv):,} observations")


## STEP 9 — Replicate Table 1 Statistics

Generate descriptive statistics (Mean, Median, SD) for both **All Firms** and **Survivors**.

**Trimming approach:** Rather than dropping rows from the dataset, we compute trimmed statistics (1st/99th percentile) per variable within the summary function. This preserves sample size while handling outliers.


In [ ]:
# Variables to include in Table 1
vars_table1 = [
    'book_lev', 'market_lev', 'log_sales', 'mtb', 'profitability',
    'tangibility', 'cf_vol', 'ind_med_lev', 'div_payer', 'intangibles'
]

def trimmed_stats(s, q=0.01):
    """
    Compute mean, median, std after trimming tails at q and 1-q quantiles.
    This is applied per variable, not across the entire dataset.
    """
    s = s.dropna()
    if s.empty or len(s) < 10:
        return (np.nan, np.nan, np.nan)
    lo, hi = s.quantile([q, 1-q])
    s_trimmed = s[(s >= lo) & (s <= hi)]
    return (s_trimmed.mean(), s_trimmed.median(), s_trimmed.std())

def summary_table(data, q=0.01):
    """
    Generate summary statistics table with trimmed moments.
    Each variable is trimmed independently at 1%/99% by default.
    """
    out = {}
    for v in vars_table1:
        mean, med, sd = trimmed_stats(data[v], q=q)
        out[v] = {'Mean': mean, 'Median': med, 'SD': sd}
    return pd.DataFrame(out).T

# Generate tables
table_all = summary_table(df_all)
table_surv = summary_table(df_surv)


In [ ]:
print("="*80)
print("TABLE 1 REPLICATION: ALL FIRMS")
print("="*80)
print(table_all.round(3))
print(f"\nNumber of observations: {len(df_all):,}")
print(f"Number of unique firms: {df_all['gvkey'].nunique():,}")


In [ ]:
print("="*80)
print("TABLE 1 REPLICATION: SURVIVORS")
print("="*80)
print(table_surv.round(3))
print(f"\nNumber of observations: {len(df_surv):,}")
print(f"Number of unique firms: {df_surv['gvkey'].nunique():,}")


## STEP 10 — Comparison: All Firms vs Survivors

Compare the two groups side-by-side to highlight differences.


In [ ]:
# Create side-by-side comparison
comparison = pd.DataFrame({
    'All_Mean': table_all['Mean'],
    'All_Median': table_all['Median'],
    'Surv_Mean': table_surv['Mean'],
    'Surv_Median': table_surv['Median'],
    'Diff_Mean': table_surv['Mean'] - table_all['Mean']
})

print("="*80)
print("COMPARISON: ALL FIRMS vs SURVIVORS")
print("="*80)
print(comparison.round(3))
print("\nKey Observations:")
print("- Survivors are LARGER (higher log_sales)")
print("- Survivors are MORE PROFITABLE (higher profitability)")
print("- Survivors are MORE TANGIBLE (higher tangibility)")
print("- Survivors have LOWER GROWTH (lower mtb)")
print("- Survivors are MORE LEVERED (higher book & market leverage)")


## Export Results

Save the results to CSV files for further analysis or reporting.


In [ ]:
# Export summary tables
table_all.to_csv('table1_all_firms.csv')
table_surv.to_csv('table1_survivors.csv')
comparison.to_csv('table1_comparison.csv')

print("Results exported to CSV files!")
print("  - table1_all_firms.csv")
print("  - table1_survivors.csv")
print("  - table1_comparison.csv")


In [ ]:
# Close the database connection
db.close()
print("WRDS connection closed.")
